##**1. Importing the Required Libraries**

In [ ]:
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# librosa is a Python library for analyzing audio and music.
# It can be used to extract the data from the audio files we will see it later
import librosa
import torch
import librosa.display
import warnings
warnings.filterwarnings("ignore")
# to play the audio files
from IPython.display import Audio
plt.style.use('seaborn-white')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Audio Feature Extraction using wavLM**

In [ ]:
import torchaudio


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.9 MB/s eta 0:00:00


In [ ]:
from transformers import AutoProcessor, WavLMModel, Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wavlm-libri-clean-100h-base-plus")
wavlm = WavLMModel.from_pretrained("patrickvonplaten/wavlm-libri-clean-100h-base-plus")# audio file is decoded on the fly

def extract_features(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)
    input = processor(array.squeeze(), sampling_rate= sample_rate, return_tensors="pt")
    # apply the model to the input array from wav
    with torch.no_grad():
       outputs = wavlm(**input)
    # extract last hidden state, compute average, convert to numpy
    last_hidden_states = outputs.last_hidden_state.squeeze().mean(axis=0).numpy()
    return last_hidden_states

## **2.1 Training Dataset**

## **2.1.1 Dataset loading**

In [ ]:
csv_file_path = '/content/drive/MyDrive/Audio_Data/annotations/annotations/Bhojpuri_train.csv'
# Read the CSV file into a DataFrame
bhoj_train = pd.read_csv(csv_file_path)
bhoj_train.head()

,filename,label
0,Abuse_1.wav,No
1,Abuse_2.wav,Yes
2,Abuse_4.wav,Yes
3,Abuse_5.wav,Yes
4,Abuse_6.wav,Yes


In [ ]:
bhoj_train.shape

(759, 2)

## **2.1.2 Feature Extraction**

In [ ]:
image_directory_bhoj = '/content/drive/MyDrive/Audio_Data/Indian_Language_Audio_Dataset/SC_audio_Bhojpuri/'
bhoj_train['filename']= image_directory_bhoj + "/" + bhoj_train['filename']
bhoj_train.head()

,filename,label
0,/content/drive/MyDrive/Audio_Data/Indian_Langu...,No
1,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
2,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
3,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
4,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes


In [ ]:
#save embedding in Numpy


In [ ]:
bhoj_train_embeddings = []
for i in range(len(bhoj_train)):
    features = extract_features(bhoj_train['filename'][i])
    bhoj_train_embeddings.append(features)

bhoj_train_embeddings = np.array(bhoj_train_embeddings)
print(bhoj_train_embeddings.shape)

(759, 768)


In [ ]:
np.save('/content/drive/MyDrive/Audio_Data/Output/bhoj_train_embeddings.npy',bhoj_train_embeddings)

print("bhoj_train_embeddings.npy file is saved")


bhoj_train_embeddings.npy file is saved


## **2.2 Testing Dataset**

## **2.2.1 Dataset Loading**

In [ ]:
csv_file_path_test = '/content/drive/MyDrive/Audio_Data/annotations/annotations/Bhojpuri_test.csv'
# Read the CSV file into a DataFrame
bhoj_test = pd.read_csv(csv_file_path_test)
bhoj_test.head()

,filename,label
0,Abuse_0.wav,No
1,Abuse_3.wav,Yes
2,Abuse_7.wav,Yes
3,Abuse_11.wav,Yes
4,Abuse_14.wav,Yes


In [ ]:
bhoj_test['filename']= image_directory_bhoj + "/" + bhoj_test['filename']
bhoj_test.head()

,filename,label
0,/content/drive/MyDrive/Audio_Data/Indian_Langu...,No
1,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
2,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
3,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes
4,/content/drive/MyDrive/Audio_Data/Indian_Langu...,Yes


In [ ]:
feats_test = np.array(extract_features(bhoj_test['filename'][0]))
feats_test.shape

(768,)

## **2.2.2 Feature Extraction**

In [ ]:
bhoj_test_embeddings = []
for i in range(len(bhoj_test)):
    features_test = extract_features(bhoj_test['filename'][i])
    bhoj_test_embeddings.append(features_test)

bhoj_test_embeddings = np.array(bhoj_test_embeddings)
print(bhoj_test_embeddings.shape)

(336, 768)


In [ ]:
np.save('/content/drive/MyDrive/Audio_Data/Output/bhoj_test_embeddings.npy',bhoj_test_embeddings)

print("bhoj_test_embeddings.npy file is saved")

bhoj_test_embeddings.npy file is saved
